In [ ]:
import pandas as pd
import regex as re

In [ ]:
years = range(2017,2022)
naics_level = [2,4,6]

In [ ]:
state_dict = {
    "AL": "ALABAMA",
    "AK": "ALASKA",
    "AZ": "ARIZONA",
    "AR": "ARKANSAS",
    "CA": "CALIFORNIA",
    "CO": "COLORADO",
    "CT": "CONNECTICUT",
    "DE": "DELAWARE",
    "FL": "FLORIDA",
    "GA": "GEORGIA",
    "HI": "HAWAII",
    "ID": "IDAHO",
    "IL": "ILLINOIS",
    "IN": "INDIANA",
    "IA": "IOWA",
    "KS": "KANSAS",
    "KY": "KENTUCKY",
    "LA": "LOUISIANA",
    "ME": "MAINE",
    "MD": "MARYLAND",
    "MA": "MASSACHUSETTS",
    "MI": "MICHIGAN",
    "MN": "MINNESOTA",
    "MS": "MISSISSIPPI",
    "MO": "MISSOURI",
    "MT": "MONTANA",
    "NE": "NEBRASKA",
    "NV": "NEVADA",
    "NH": "NEW HAMPSHIRE",
    "NJ": "NEW JERSEY",
    "NM": "NEW MEXICO",
    "NY": "NEW YORK",
    "NC": "NORTH CAROLINA",
    "ND": "NORTH DAKOTA",
    "OH": "OHIO",
    "OK": "OKLAHOMA",
    "OR": "OREGON",
    "PA": "PENNSYLVANIA",
    "RI": "RHODE ISLAND",
    "SC": "SOUTH CAROLINA",
    "SD": "SOUTH DAKOTA",
    "TN": "TENNESSEE",
    "TX": "TEXAS",
    "UT": "UTAH",
    "VT": "VERMONT",
    "VA": "VIRGINIA",
    "WA": "WASHINGTON",
    "WV": "WEST VIRGINIA",
    "WI": "WISCONSIN",
    "WY": "WYOMING"
}

In [ ]:
target_url = f"https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/targets/bees-targets.csv"
target_df = pd.read_csv(target_url)

In [ ]:
def rename_columns(df, year):
    rename_mapping = {}
    for column in df.columns:
      if column not in df.columns[:2]:
          new_column_name = column + f'-{year}'
          rename_mapping[column] = new_column_name

    df.rename(columns=rename_mapping, inplace=True)

In [ ]:
for state in state_dict:
  data = {}
  for year in years:
    url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics2/US/counties/{year}/US-{state}-training-naics2-counties-{year}.csv"
    data[year] = pd.read_csv(url)
    rename_columns(data[year], year)

  merged_df_feature = pd.merge(data[2017], data[2018], on=['Fips', 'Name'], how='inner')
  for year in range(2019,2022):
    merged_df_feature = pd.merge(merged_df_feature, data[year], on=['Fips', 'Name'], how='inner')

  cols = merged_df_feature.columns.tolist()
  cols = cols[:2] + sorted(cols[2:])
  merged_df_feature = merged_df_feature[cols].rename(columns={"Name": "County"})

  merged_df = pd.merge(merged_df_feature, target_df[target_df["State"]==state_dict[state]], on=["Fips","County"], how="inner")
  merged_df.drop(columns=['Unnamed: 0','Name','State','State ANSI', 'County ANSI', '2007_increase','2012_increase','2017_increase'], axis=1, inplace=True)
  merged_df.to_csv(f"{state}-all-years.csv")
  # print(merged_df)
  # break

    Fips             County  Emp-11-2017  Emp-11-2018  Emp-11-2019  \
0   1099      Monroe County        207.0        217.0        192.0   
1   1079    Lawrence County          NaN          NaN          NaN   
2   1081         Lee County         79.0         83.0         43.0   
3   1093      Marion County         71.0         68.0         63.0   
4   1107     Pickens County         89.0         95.0         92.0   
..   ...                ...          ...          ...          ...   
60  1001     Autauga County         93.0         93.0         86.0   
61  1123  Tallapoosa County         62.0         61.0         71.0   
62  1061      Geneva County          NaN          NaN          NaN   
63  1131      Wilcox County         97.0         91.0         74.0   
64  1037       Coosa County         81.0         73.0         76.0   

    Emp-11-2020  Emp-11-2021  Emp-21-2017  Emp-21-2018  Emp-21-2019  ...  \
0         174.0        200.0          NaN          NaN         28.0  ...   
1      